**This workbook demonstrates the application of a simple Gamma distribution to evaluating molar fractions and average MW's over a range of defined LMW's, using an analytical form of the integrated Gamma and mass weighted Gamma distribution

In [1]:
import scipy as scipy
from scipy.special import gamma
import math

In [26]:
# Incomplete gamma function as evaluated by Wolfram - differs from native scipy gammainc(a,x)
# Test is incomp_gamma(0.01,0.1) = 1.803241356902546, which is same as https://www.wolframalpha.com/input/?i=gamma(0.01,0.1)
def incomp_gamma(a,x):
    return gamma(a)*(1 - scipy.special.gammainc(a,x))

# Following equation references from "C7+ Characterization of Related Equilibrium Fluids Using the Gamma Distribution"
# By Whitson, Anderson and Soreide

# Calculating the probability and cumulative probability density function
# Probability density function - Eq 1.
def pM(M, eta, alpha, beta, precalc_gamma):
    return (M - eta)**(alpha-1)*math.exp(-(M-eta)/beta)/(beta**alpha*precalc_gamma)

def Cum_pM(LMW, eta, alpha, plus_MW):
    # Returns analytcally integrated the pM fuction between eta on the lower side, to M on the high side
    # https://www.wolframalpha.com/input/?i=integrate+(x-a)%5Eb*exp(-(x-a)%2Fc)%2Fd
    beta = (plus_MW - eta)/alpha
    
    if LMW <= eta:
        return 0
    a = eta
    b = alpha-1
    c = beta
    d = beta**alpha*scipy.special.gamma(alpha)
    return 1-c*(LMW-a)**b*((LMW-a)/c)**-b*incomp_gamma(b+1,(LMW-a)/c)/d

# And now the mass weighted probability and cumulative mass weighted probability density functions 

# Mass weighted Probability density function.
def pMM(M, eta, alpha, beta, precalc_gamma):
    return M*(M - eta)**(alpha-1)*math.exp(-(M-eta)/beta)/(beta**alpha*precalc_gamma)

def Cum_pMM(LMW, eta, alpha, plus_MW):
    # Analytically integrates the pMM fuction between eta on the lower side, to LMW on the high side
    #https://www.wolframalpha.com/input/?i=integrate+(x-a)%5Eb*exp(-(x-a)%2Fc)*x%2Fd
    beta = (plus_MW - eta)/alpha
    
    if LMW <= eta:
        return 0
    x = LMW
    a = eta
    b = alpha-1
    c = beta
    d = beta**alpha*scipy.special.gamma(alpha)
    return beta*alpha+eta-((c*(x - a)**b * (a* incomp_gamma(1 + b, ( x-a )/c) + c* incomp_gamma(2 + b, (-a + x)/c)))/(d* ((x-a)/c)**b))

In [45]:
# And now test the code with a simple Gamma distribution

eta = 90
alpha = 0.75
plus_MW = 180
# Define a list of LMW's
LMW = [100, 115, 130, 145, 160, 175, 180, 195, 210, 225]

# Calculate the cumulative probability and mass weighted probability
cum_prob = [0]
cum_mass_prob = [0]
for M in LMW:
    cum_prob.append(Cum_pM(M, eta, alpha, plus_MW))
    cum_mass_prob.append(Cum_pMM(M, eta, alpha, plus_MW))
cum_prob.append(1)
cum_mass_prob.append(plus_MW)

# Calculate incremental molar fraction by subtracting previous values
Molar_frac = [cum_prob[i]-cum_prob[i-1] for i in range(1,len(cum_prob))]

# And average MW's in the LMW bins
Avg_MWs = [(cum_mass_prob[i]-cum_mass_prob[i-1])/Molar_frac[i-1] for i in range(1,len(cum_prob))]

sum_mass = 0
# Check total mole weighted masses add to target C7+ MW
for i in range(len(Molar_frac)):
    sum_mass += Molar_frac[i]*Avg_MWs[i]
    
print('Comp,  Mole Frac,   Avg MW,   Mass')
for i in range(len(Molar_frac)):
    print(i,'     ',round(Molar_frac[i],4),',  ',round(Avg_MWs[i],4),',  ', round(Molar_frac[i]*Avg_MWs[i],4) )

print(' ')
print('Total:   ',sum(Molar_frac),',         -       ,',sum_mass)

Comp,  Mole Frac,   Avg MW,   Mass
0       0.1629 ,   94.2117 ,   15.3461
1       0.1446 ,   107.0631 ,   15.4777
2       0.1084 ,   122.1977 ,   13.2413
3       0.0868 ,   137.2445 ,   11.9137
4       0.0715 ,   152.2686 ,   10.883
5       0.0597 ,   167.2832 ,   9.995
6       0.0177 ,   177.4767 ,   3.1502
7       0.0477 ,   187.2957 ,   8.9418
8       0.0406 ,   202.3021 ,   8.2227
9       0.0348 ,   217.307 ,   7.5539
10       0.2253 ,   334.1699 ,   75.2746
 
Total:    1.0 ,         -       , 180.0
